In [1]:
import os
import fnmatch
import itertools
import numpy as np
from numpy import linalg as LA
from ivisual import *

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
################ This code allows choosing the cells to view #######
##### First load data: poss[ibox,cell_idx 1:3, idx_atom 1:16, xyz 1:3]
##############################################


######load spin configuration files
##Input: no. of atom per cell and supercell and no of files
natcell,spdim,subbox,ibox=16,[6,6,6],[3,3,3],5
#for GdZrO:16 atom in a cell, 6*6*6 supercell, size of sc to view, nb of files

atpos=np.zeros([natcell,3])#store the atom positions in a cell
spins=np.zeros(np.hstack([ibox,spdim,natcell,9]))#store the spins in a supercell for 5 runs

filepath = r'D:\9_Program files\SpinVert\gdzro_ill\gd_ill_60mk-20k_with_paramag/'
filename = '10k_0p994_s3_wl2p41_p3500i_spins*.txt'
filepath = r'D:\9_Program files\SpinVert\0_spinvert_program\spinvert_with_modifed\examples\spinice/'
filename = 'spinice_spins_01.txt'
# Read the 16 atom positions
ii=0
for file in os.listdir(filepath):
    if fnmatch.fnmatch(file, filename):   
        print file
        if ii==0:
            with open(os.path.join(filepath,file), 'rU') as f:
                i=0
                for line in f:
                    line=line.rstrip('\n').split()# without giving delimenter, it will treat conseutive space as one
                    if line[0]=="SITE":
                        atpos[i,:]=line[1:]#automatically convert string to np.array
                        i=i+1

# local [111] axes for the 16 atoms
aniso = np.array([[1,1,1],[1,1,1],[1,1,1],[1,1,1],
                  [1,-1,-1],[1,-1,-1],[1,-1,-1],[1,-1,-1],
                  [-1,1,-1],[-1,1,-1],[-1,1,-1],[-1,1,-1],
                  [-1,-1,1],[-1,-1,1],[-1,-1,1],[-1,-1,1]])/np.sqrt(3.)/4.0

# Read the all the spin orientations with doing Sx*local11,Sy*local11,Sz*local11
ii=0 #index for different files
jj=0
for file in os.listdir(filepath):
    if fnmatch.fnmatch(file, filename):   
        print file
        with open(os.path.join(filepath,file), 'rU') as f:
            for line in f:
                line=line.rstrip('\n').split()# without giving delimenter, it will treat conseutive space as one
                if line[0]=="SPIN":
                    idxsp=np.array(line[1:5],dtype='int32')# atom index, superCell idx:[i,j,k]
                    sxyz = np.array(line[5:8],dtype='float') # spin orientatoins
                    # For Heisenberg: Sx*local,Sy*local,Sz*local
                    #spins[ii,idxsp[1],idxsp[2],idxsp[3],idxsp[0]-1,:]=np.hstack(
                        #[sxyz[0]*aniso[idxsp[0]-1,:],sxyz[1]*aniso[idxsp[0]-1,:],sxyz[2]*aniso[idxsp[0]-1,:]])
                    #print np.dot(sxyz, aniso[idxsp[0]-1,:])
                    # For Spin ice: only the for 3 elements is needed Si*localZ_i=+/-1 * localZ_i,  the left in not needed
                    spins[ii,idxsp[1],idxsp[2],idxsp[3],idxsp[0]-1,:]=np.hstack(
                        [4.*np.dot(sxyz, aniso[idxsp[0]-1,:])*aniso[idxsp[0]-1,:],sxyz[1]*aniso[idxsp[0]-1,:],sxyz[2]*aniso[idxsp[0]-1,:]])
                    if jj<1:
                        print line[5:8], spins[ii,idxsp[1],idxsp[2],idxsp[3],idxsp[0]-1,:]
                        jj += 1
        ii=ii+1

        
##### find the nearest Gd-Gd neighers
bonds = np.zeros([5,1000])#store the 30 bonds in the unitCell: atom1,atom2,pos[a2]-pos[a1]
idx = 0
for i in np.arange(0,len(atpos)-1,1):
    for ii in np.arange(i+1,len(atpos),1):
        distance = LA.norm(atpos[i]-atpos[ii])
        if distance==np.sqrt(2)/4 and ~np.in1d(ii,bonds[0,np.where(bonds[1,:]==i)])[0]:
            bonds[:,idx]=np.append(np.array([i, ii]),atpos[ii]-atpos[i])
            idx +=1
            
print atpos, spins.shape

spinice_spins_01.txt
spinice_spins_01.txt
['-0.57735026918962573', '-0.57735026918962573', '-0.57735026918962573'] [-0.14433757 -0.14433757 -0.14433757 -0.08333333 -0.08333333 -0.08333333
 -0.08333333 -0.08333333 -0.08333333]
[[ 0.5   0.5   0.5 ]
 [ 0.5   0.    0.  ]
 [ 0.    0.    0.5 ]
 [ 0.    0.5   0.  ]
 [ 0.5   0.75  0.75]
 [ 0.5   0.25  0.25]
 [ 0.    0.25  0.75]
 [ 0.    0.75  0.25]
 [ 0.75  0.5   0.75]
 [ 0.75  0.    0.25]
 [ 0.25  0.    0.75]
 [ 0.25  0.5   0.25]
 [ 0.75  0.25  0.  ]
 [ 0.75  0.75  0.5 ]
 [ 0.25  0.75  0.  ]
 [ 0.25  0.25  0.5 ]] (5L, 6L, 6L, 6L, 16L, 9L)


In [3]:
# Diamond lattic: center of the tetrahedra (5) + and three from O+0.25[111] 
# tetrahedra within a unit cell
tetra = np.array([[0,4,8,13],
                  [1,5,9,12],
                  [2,6,10,15],
                  [3,7,11,14],
                  [0,5,11,15]]) 

print atpos[tetra[0,:]]
diamond = np.vstack(atpos[tetra,:].sum(axis=2)/4.)#.flatten()
print diamond, diamond.shape

print tetra.flatten()
print tetra

o8b = atpos[tetra,:].sum(axis=1)/4.
theOther3o = o8b[0,:] - np.array([[1,-1,-1], [-1,1,-1], [-1,-1,1]])/4.
o8b = np.vstack([o8b,theOther3o])
print o8b

o8b = np.array([ [ 0.625,  0.625,  0.625],
                 [ 0.625,  0.125,  0.125],
                 [ 0.125,  0.125,  0.625],
                 [ 0.125,  0.625,  0.125],
                 [ 0.375,  0.375,  0.375],
                 [ 0.375,  0.875,  0.875],
                 [ 0.875,  0.375,  0.875],
                 [ 0.875,  0.875,  0.375]])

[[ 0.5   0.5   0.5 ]
 [ 0.5   0.75  0.75]
 [ 0.75  0.5   0.75]
 [ 0.75  0.75  0.5 ]]
[[ 0.375  0.5    0.5    0.5  ]
 [ 0.125  0.25   0.25   0.25 ]
 [ 0.125  0.25   0.25   0.25 ]
 [ 0.125  0.25   0.25   0.25 ]
 [ 0.375  0.25   0.25   0.25 ]] (5L, 4L)
[ 0  4  8 13  1  5  9 12  2  6 10 15  3  7 11 14  0  5 11 15]
[[ 0  4  8 13]
 [ 1  5  9 12]
 [ 2  6 10 15]
 [ 3  7 11 14]
 [ 0  5 11 15]]
[[ 0.625  0.625  0.625]
 [ 0.625  0.125  0.125]
 [ 0.125  0.125  0.625]
 [ 0.125  0.625  0.125]
 [ 0.375  0.375  0.375]
 [ 0.375  0.875  0.875]
 [ 0.875  0.375  0.875]
 [ 0.875  0.875  0.375]]


In [4]:
# Find the neigbors of oxygen ions on the first unit cell
# create a 333 supercell and search neighbor for the O ions in the centeral cell
permu=np.array(list(itertools.product([0,1,-1], repeat=3) ) )
allpos = np.vstack([atpos+item for item in permu])

orBondsIdx = np.zeros([32,2],dtype=int) #store the 32 R-O(8b) bonds in the unitCell: idx(O8b),idx(R),
orBonds = np.zeros([32,8])#store the 32 R-O(8b) bonds in the unitCell: idx(O8b),idx(R), dL, pos[a2]-pos[a1]
idx = 0
for i in np.arange(0,len(o8b),1):
    for ii in np.arange(0,len(allpos),1):
        distance = LA.norm(o8b[i]-allpos[ii])
        if distance==np.sqrt(3/64.):
            ridx = np.remainder(ii,16)
            orBondsIdx[idx,:] = [i*1., ridx]
            orBonds[idx,:]=np.hstack( [ [i,ridx], np.trunc(allpos[ii]), allpos[ii]-o8b[i] ] )
            idx +=  1

#print orBondsIdx
print orBonds

[[  0.      0.      0.      0.      0.     -0.125  -0.125  -0.125]
 [  0.      4.      0.      0.      0.     -0.125   0.125   0.125]
 [  0.      8.      0.      0.      0.      0.125  -0.125   0.125]
 [  0.     13.      0.      0.      0.      0.125   0.125  -0.125]
 [  1.      1.      0.      0.      0.     -0.125  -0.125  -0.125]
 [  1.      5.      0.      0.      0.     -0.125   0.125   0.125]
 [  1.      9.      0.      0.      0.      0.125  -0.125   0.125]
 [  1.     12.      0.      0.      0.      0.125   0.125  -0.125]
 [  2.      2.      0.      0.      0.     -0.125  -0.125  -0.125]
 [  2.      6.      0.      0.      0.     -0.125   0.125   0.125]
 [  2.     10.      0.      0.      0.      0.125  -0.125   0.125]
 [  2.     15.      0.      0.      0.      0.125   0.125  -0.125]
 [  3.      3.      0.      0.      0.     -0.125  -0.125  -0.125]
 [  3.      7.      0.      0.      0.     -0.125   0.125   0.125]
 [  3.     11.      0.      0.      0.      0.125  -0.125   0.

In [5]:
############### Chekcthe found O neigiebors and  aniso axes #################
# local [111] axes for the 16 atoms
aniso0 = np.array([[1,1,1],[1,1,1],[1,1,1],[1,1,1],
                  [1,-1,-1],[1,-1,-1],[1,-1,-1],[1,-1,-1],
                  [-1,1,-1],[-1,1,-1],[-1,1,-1],[-1,1,-1],
                  [-1,-1,1],[-1,-1,1],[-1,-1,1],[-1,-1,1]])/2.

In [7]:
scene1 = canvas(title='MagStr', width=500, height=500,x=500,y=500,center=(0.5,0.5,0.5),
                 background=color.white,exit=False)

scene1.autocenter = True # why not working
scene1.autoscale = True 
spinl = 0.2; atomr = 0.5; cylr = 0.005

# Plot the Gd atoms, easy axes and the bonds
for i in tetra.flatten():
    #ii = i+16
    pointer = arrow(pos=atpos[i]-spinl*aniso0[i]/2, axis=spinl*aniso0[i],shaftwidth=0.01) # easy axis
    pointer = sphere(pos=atpos[i], color=color.yellow, radius=atomr*0.05) # atoms
    pointer = label(pos=atpos[i]+(0,0,0.1), text=str(i%16))    

for i in range(bonds.shape[1]):
    pointer = cylinder(pos=atpos[bonds[0,i]], axis=bonds[2:,i], radius=cylr,materials=materials.wood)

# Plot the O atoms and the surrouding Gd (slightly shfited along local 111, to compare with the Gd)
for i in range(len(o8b)):
    pointer = sphere(pos=o8b[i], color=color.hsv_to_rgb((0.6667+i*0.333/8,1,0.5))  , radius=atomr*0.2) # color: blue-red
    around = orBonds[orBonds[:,0]==i]
    for ii in range(len(around)):
        pointer = sphere(pos=o8b[i]+around[ii,5:]*0.8, color=color.hsv_to_rgb((0.6667+i*0.333/8,1,0.5)),
                         radius=atomr*0.02)
        
scene1.lights = []
scene1.ambient= 2


<IPython.core.display.Javascript object>

C:\Users\kew\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


## Now prepared the spins of diamond lattice and save

In [20]:
# Create diamond or o8b superlattice
sc6 = np.zeros([8*6**3,7]) # atIdx0-32, cell index x,y,z, atom index x, y, z
idx = 0
ibox = 0
for i in xrange(6):
    for j in xrange(6):
        for k in xrange(6):
            for l in np.arange(0,8):
                tetra = np.array(orBonds[orBonds[:,0]==l,1:5],dtype='int32')
                #print tetra
                idxCell = np.array([i,j,k] + tetra[:,1:4])
                idxCell = np.where(idxCell==6,0,idxCell)
                #print idxCell
                # 0:3 for Px, 3:6 for Px, 6 :9 for Pz
                sumTetra = spins[ibox,idxCell[:,0],idxCell[:,1],idxCell[:,2],tetra[:,0],0:3].sum(axis=0) 
                sc6[idx,:] = np.hstack((np.array([l+1,i,j,k]),sumTetra))
                idx = idx + 1

In [ ]:
idxCell = np.array([[-1,0,0],[0,1, 0]])
[1,2,3]+ idxCell
np.where(idxCell==-1,0,idxCell)

In [21]:
# Save
path = r'D:\9_Program files\SpinVert\gdzro_ill\gd_ill_60mk-20k_with_paramag\diamond_correlations\spinice_spins_01_diamond.txt'

# second save super lattice
f = open(path, 'w+') # append
f.write('TITLE spin \n')
f.write('CELL 10.440 10.440 10.440 90.00 90.00 90.00\n')

names  = np.repeat('SITE',8)
outputs = map(lambda x: "%s  %f %f %f " % (x[0],x[1][0],x[1][1],x[1][2]),
              zip(names,o8b)) 
data = '\n'.join(outputs)+'\n' # Concat all strings in list, separated by line break
f.write(data)

f.write('BOX 6 6 6\nFORM_FACTOR_J0 0 0 0\n\
PROPOSED_MOVES 0\nACCEPTED_MOVES 0\nWEIGHT 3\nCHI_SQUARED 14\nR_FACTOR 3.4\nSCALE 72\n\
FLAT_BACKGROUND 0.0\nLINEAR_BACKGROUND 0\n')

names  = np.repeat('SPIN',8*6**3).astype('str')
outputs = map(lambda x: "%s %i %i %i %i %f %f %f " % (x[0],x[1][0],x[1][1],x[1][2],x[1][3],x[1][4],x[1][5],x[1][6]),
              zip(names, sc6)) 
data = '\n'.join(outputs) # Concat all strings in list, separated by line break
f.write(data)
f.close()